In [1]:
import pandas as pd #Đọc file dữ liệu

student = pd.read_csv("data_clean\student_profile_clean.csv")
club_reg = pd.read_csv("data_clean\club_registration_clean.csv")
event = pd.read_csv("data_clean\event_attendance_clean.csv")


student_df = pd.DataFrame(student)
club_df = pd.DataFrame(club_reg)
event_df = pd.DataFrame(event)

<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\c'
<>:5: SyntaxWarning: invalid escape sequence '\e'
<>:3: SyntaxWarning: invalid escape sequence '\s'
<>:4: SyntaxWarning: invalid escape sequence '\c'
<>:5: SyntaxWarning: invalid escape sequence '\e'
C:\Users\admin\AppData\Local\Temp\ipykernel_1504\3909475154.py:3: SyntaxWarning: invalid escape sequence '\s'
  student = pd.read_csv("data_clean\student_profile_clean.csv")
C:\Users\admin\AppData\Local\Temp\ipykernel_1504\3909475154.py:4: SyntaxWarning: invalid escape sequence '\c'
  club_reg = pd.read_csv("data_clean\club_registration_clean.csv")
C:\Users\admin\AppData\Local\Temp\ipykernel_1504\3909475154.py:5: SyntaxWarning: invalid escape sequence '\e'
  event = pd.read_csv("data_clean\event_attendance_clean.csv")


In [2]:
student_club = student.merge( #Gộp theo student id và phân tích số clb theo lớp khóa
    club_df,
    on="student_id",
    how="left"
)
print(student_club)

    student_id     full_name class_cohort          club_id  join_count
0       SV2000    Tran Thi B        K18A2           CLB_TN         0.0
1       SV2000    Tran Thi B        K18A2      CLB_AM_NHAC         0.0
2       SV2001  Nguyen Van A        K17A1              NaN         NaN
3       SV2002   Hoang Van E        K19A2           CLB_TN         2.0
4       SV2003      Le Van C        K18A1       CLB_KN_MEM         2.0
..         ...           ...          ...              ...         ...
153     SV2078    Tran Thi B        K18A1       CLB_KN_MEM         3.0
154     SV2078    Tran Thi B        K18A1  CLB_KY_NANG_MEM         5.0
155     SV2079    Tran Thi B        K18A1       CLB_AMNHAC         1.0
156     SV2079    Tran Thi B        K18A1     CLB_THE_THAO         1.0
157     SV2079    Tran Thi B        K18A1       CLB_KN_MEM         2.0

[158 rows x 5 columns]


In [3]:
event_full = event.merge(
    student_df,
    on="student_id", #Merger theo student id
    how="left"
)
print(event_full)
event_full_1 = event_full.merge(
    club_df,
    on=["student_id", "club_id"],
    how="left",
    indicator=True

)
print(event_full_1)

    event_id              event_name student_id          club_id  \
0      EV300            Teambuilding     SV2015       CLB_AMNHAC   
1      EV301     Hoi Thao Dinh Huong     SV2012     CLB_THE_THAO   
2      EV302            Teambuilding     SV2050  CLB_KY_NANG_MEM   
3      EV303            Ngay Hoi Clb     SV2056       CLB_AMNHAC   
4      EV304            Teambuilding     SV2018     CLB_THE_THAO   
..       ...                     ...        ...              ...   
195    EV495     Hoi Thao Dinh Huong     SV2013  CLB_KY_NANG_MEM   
196    EV496     Hoi Thao Dinh Huong     SV2078       CLB_KN_MEM   
197    EV497            Ngay Hoi Clb     SV2020  CLB_KY_NANG_MEM   
198    EV498  Chuoi Hoi Thao Ky Nang     SV2001       CLB_KN_MEM   
199    EV499            Ngay Hoi Clb     SV2009      CLB_AM_NHAC   

    attendance_status         checkin_time     full_name class_cohort  
0              Absent  2024-01-03 15:13:00    Pham Thi D        K18A1  
1              Absent                  

In [4]:
sv_khong_tham_gia_club = event_full_1[
    event_full_1["_merge"] == "left_only"
]

print(
    sv_khong_tham_gia_club[
        ["student_id", "club_id", "event_id"]
    ]
)

    student_id          club_id event_id
0       SV2015       CLB_AMNHAC    EV300
1       SV2012     CLB_THE_THAO    EV301
4       SV2018     CLB_THE_THAO    EV304
5       SV2065       CLB_KN_MEM    EV305
6       SV2078     CLB_THE_THAO    EV306
..         ...              ...      ...
200     SV2031       CLB_AMNHAC    EV494
201     SV2013  CLB_KY_NANG_MEM    EV495
203     SV2020  CLB_KY_NANG_MEM    EV497
204     SV2001       CLB_KN_MEM    EV498
205     SV2009      CLB_AM_NHAC    EV499

[146 rows x 3 columns]


In [5]:
# ===== Chuẩn hóa & tạo is_attended =====
event["student_id"] = event["student_id"].str.strip().str.upper()
event["club_id"] = event["club_id"].str.strip().str.upper()

event["attendance_status"] = (
    event["attendance_status"]
    .astype(str)
    .str.strip()
    .str.lower()
)

event["is_attended"] = event["attendance_status"].isin(
    ["có mặt", "co mat", "cm", "present"]
)


reg_count = club_reg.groupby("club_id").agg(
    nguoi_dang_ki=("student_id", "nunique")
).reset_index()
event_attended = event[event["is_attended"]]

event_of_registered = event_attended.merge(
    club_reg,
    on=["student_id", "club_id"],
    how="inner"
)

attend_count = event_of_registered.groupby("club_id").agg(
    nguoi_tham_gia=("student_id", "nunique")
).reset_index()

thong_ke_clb = reg_count.merge(
    attend_count,
    on="club_id",
    how="left"
)

thong_ke_clb["nguoi_tham_gia"] = thong_ke_clb["nguoi_tham_gia"].fillna(0)

thong_ke_clb["attendance_rate"] = (
    thong_ke_clb["nguoi_tham_gia"] /
    thong_ke_clb["nguoi_dang_ki"]
)

clb_ty_le_tham_gia_thap = thong_ke_clb[
    (thong_ke_clb["nguoi_dang_ki"] >= 20) &
    (thong_ke_clb["attendance_rate"] < 0.3)
]

print(
    clb_ty_le_tham_gia_thap[
        ["club_id", "nguoi_dang_ki", "nguoi_tham_gia", "attendance_rate"]
    ]
)


        club_id  nguoi_dang_ki  nguoi_tham_gia  attendance_rate
0    CLB_AMNHAC             34               4         0.117647
4  CLB_THE_THAO             28               4         0.142857
5        CLB_TN             22               2         0.090909
